# Relative Attributes using Rank SVM

### Import scipy libraries

In [13]:
import numpy as np
import scipy
import scipy.io as sio
from scipy.sparse import csr_matrix
from scipy.optimize import least_squares

* Compute the objective function, its gradient and the set of support vectors.
* Here all the comparisions are made for single attribute at a time.
* Also here out is maximum of 0 or (1 - A*X*W) where A is a sparse 2D matrix with rows denoting as pair for similar or open and each row has 2 elements as -1 and 1 at index i,j such that the comparision is made b/w image i and image j.
* A*X*W is a 1D matrix which contatins the difference of the attribute value of image i and j.

In [14]:
def object_fun_linear(w, C, out, n0, A, X):
    out[0:n0] = np.maximum(np.zeros([n0, 1]), out[0:n0])
    obj = np.sum(np.multiply(C, np.multiply(out, out))) / 2.0 + np.dot(np.transpose(w), w) / 2.0
    grad = w - np.transpose(np.transpose(np.multiply(C, out)) * A * X)
    sv = out[0:n0] > 0, abs(out[n0:]) > 0
    return obj[0, 0], grad, sv

* Here as we are using SVM so optimization is done with hessian matrix using Newton's method.
* This fxn computes the Hessian times a given vector x (support vectors calculated using object_fun_linear).

In [15]:
def hessian_mult(w, sv, C, grad, n0, A, X):
    w = np.transpose(np.matrix(w))
    y = w
    z = np.multiply(np.multiply(C, sv), A * (X * w))
    y += np.transpose(((np.transpose(z) * A) * X)) + grad
    y = y.A1
    return y

* Now we will find a line in the direction of d using our current 'w' matrix and then we will update 'w' using that solution.
* Here 1D Newton minimization is done.
* g = The gradient along the line.
* h = The second derivative along the line.
* Using g, h and taking newton step in that direction.

In [16]:
def line_search_linear(w, d, out, C, n0 ,A, X):
    t = 0
    Xd = A * (X * d)

    while 1:
        out2 = out - t * Xd
        sv = np.nonzero( scipy.vstack(( out2[0:n0] > 0, abs(out2[n0:]) > 0 )) )[0]
        g = np.transpose(w) * d + t * np.transpose(d) * d - np.transpose(np.multiply(C[sv], out2[sv])) * Xd[sv]
        h = np.transpose(d) * d + np.transpose(Xd[sv]) * np.multiply(Xd[sv], C[sv])
        g, h = g[0, 0], h[0, 0]
        t -= g / h
        if g * g / h < 1e-8: break
    out = out2
    return t, out

* X contains the training inputs / feature vectors and is an n x d matrix (n = number of images, d is dimensionality of each image i.e. features).
* O is a sparse p x n matrix, where p is the number of preference ordered pairs. Each row of O should contain exactly one +1 and one -1. If the Pth preference pair says i > j (strength of attribute in image i is greater than strength of attribute in image j) then O(p,i) = 1 and O(p,j) = -1
* S is a sparse p x n matrix, where p is the number of preference unordered pairs that have similar strengths of the attribute. Each row of S should contain exactly one +1 and one -1.
* C is a vector of training error penalizations (one for each preference pair).
* out = 1 - A*X*W
* Then finding support vectors usnig X, S, O, C, and then update w using the line found using 'line_search_linear' fxn.

In [17]:
def rank_svm(X_, S_, O_):
    max_itr, prec = 10, 1e-8

    X = X_; A = O_; B = S_;
    n0, d = A.shape[0], X.shape[1]
    w = scipy.matrix(scipy.zeros([d, 1]))

    itr = 0
    C = 0.1*scipy.ones([A.shape[0]+B.shape[0], 1])

    out1 = scipy.matrix(scipy.vstack( (scipy.ones([A.shape[0], 1]), scipy.zeros([B.shape[0], 1])) ))
    A = scipy.sparse.vstack((A, B))
    out = out1 - A * (X * w)
    
    while 1:
        itr += 1
        if itr > max_itr:
            print("Maximum number of Newton steps reached")
            break


        obj, grad, sv = object_fun_linear(w, C, out, n0, A, X)
        
        sv = scipy.vstack(sv)
        res = least_squares(hessian_mult, np.zeros(w.shape[0]), ftol = 1e-8, xtol = 1e-8, gtol = 1e-8, args = (sv, C, grad, n0, A, X))
        
        step = np.transpose(np.matrix(res.x))
        t, out = line_search_linear(w, step, out, C, n0, A, X)
        w += t * step;

        check = - np.transpose(step) * grad
        check = check[0, 0]
        print(check, prec*obj)
        if check < prec * obj: break

        print("Iteration = " + str(itr))
    print("Weight Matrix :")
    print(w)
    return w

* Loading data matrix containing class names, attribute names, images data(772 images reduced to 552 features), and relative ordering of images for given attributes.

In [18]:
datadict = sio.loadmat('data.mat')

In [19]:
attr_names = []
for x in datadict['attribute_names'][0]:
    attr_names.append(x[0])
datadict['attribute_names'] = attr_names

datadict['im_names'] = datadict['im_names'][0]
datadict['class_labels'] = datadict['class_labels'][:, 0]
datadict['used_for_training'] = datadict['used_for_training'][:, 0]
datadict['class_names'] = datadict['class_names'][0]

* Iterating over images and classifying images into 2 sets S, O using relative ordering. 
* If 2 images are similar we add it to S else to O.
* Then we create sparse 2D matrix with S[i][j] = -1 and S[j][i] = 1 for any pair of images i,j if they are similar.
* Similarly applying this for O matrix.

## Attribute names

In [20]:
datadict['attribute_names']

['Male',
 'White',
 'Young',
 'Smiling',
 'Chubby',
 'VisibleForehead',
 'BushyEyebrows',
 'NarrowEyes',
 'PointyNose',
 'BigLips',
 'RoundFace']

In [21]:
p = 3 # Smiling attribute
cat_ordering = datadict['relative_ordering'][p]

Sdata, Odata = [[],[],[]], [[],[],[]]
num = datadict['feat'].shape[0]
for i in range(len(datadict['class_labels'])):
    im1_lab = datadict['class_labels'][i] - 1
    for j in range(len(datadict['class_labels'][i+1:])):
        im2_lab = datadict['class_labels'][i+j+1] - 1
        if cat_ordering[im1_lab] == cat_ordering[im2_lab]:
            Sdata[0].append(len(Sdata[0])/2)
            Sdata[0].append(len(Sdata[0])/2)
            Sdata[1].append(i)
            Sdata[1].append(i+j+1)
            Sdata[2].append(-1)
            Sdata[2].append(1)

        elif cat_ordering[im1_lab] < cat_ordering[im2_lab]:
            Odata[0].append(len(Odata[0])/2)
            Odata[0].append(len(Odata[0])/2)
            Odata[1].append(i)
            Odata[1].append(i+j+1)
            Odata[2].append(-1)
            Odata[2].append(1)
        elif cat_ordering[im1_lab] > cat_ordering[im2_lab]:
            Odata[0].append(len(Odata[0])/2)
            Odata[0].append(len(Odata[0])/2)
            Odata[1].append(i)
            Odata[1].append(i+j+1)
            Odata[2].append(1)
            Odata[2].append(-1)

S = csr_matrix((Sdata[2], (Sdata[0], Sdata[1])),(int(len(Sdata[0])/2), num))
O = csr_matrix((Odata[2], (Odata[0], Odata[1])),(int(len(Odata[0])/2), num))
X = scipy.matrix(datadict['feat'])
w = rank_svm(X, S, O)

16083.5593224 0.000121055
Iteration = 1
114.578040122 3.67408412302e-05
Iteration = 2
6.0808202583 3.62167447878e-05
Iteration = 3
0.0197262559954 3.61840762647e-05
Iteration = 4
0.000538874360076 3.61839836552e-05
Iteration = 5
6.92664578045e-07 3.61839810462e-05
Weight Matrix :
[[ -1.65018776e+00]
 [  1.33968580e+00]
 [  1.04559991e+00]
 [  1.04861097e+00]
 [ -1.24701962e+00]
 [  5.17567462e-01]
 [  3.28443886e-01]
 [ -1.04781178e+00]
 [  4.01476652e-01]
 [ -1.18366460e+00]
 [ -1.17911939e+00]
 [ -1.77082122e+00]
 [  1.70579263e+00]
 [  2.10562596e-01]
 [ -4.20427918e-01]
 [  3.24491846e-01]
 [  8.84830721e-01]
 [  1.53601237e+00]
 [  5.42913847e-01]
 [  1.85849723e+00]
 [ -3.97158288e-01]
 [  1.98600427e+00]
 [ -1.26649615e+00]
 [ -4.62126589e-01]
 [ -5.09785790e-01]
 [ -6.81943672e-04]
 [ -5.44445001e-01]
 [ -2.33696926e+00]
 [  1.41991559e+00]
 [ -1.71475877e+00]
 [  2.59723983e-01]
 [ -1.07960630e+00]
 [  3.43481182e-01]
 [ -9.04499498e-02]
 [ -1.01698569e-01]
 [  6.51848658e-01]

## Comparision for Simling Attribute

In [22]:
im_names = datadict['im_names'].tolist()
im1_name = 'AlexRodriguez_38.jpg'
im2_name = 'CliveOwen_170.jpg'
im1_idx = im_names.index(im1_name)
im2_idx = im_names.index(im2_name)
print("Attrbute values for image 1 and 2 : ")
print (np.dot(X[im1_idx-1],w),np.dot(X[im2_idx-1],w))

Attrbute values for image 1 and 2 : 
[[ 0.24032394]] [[ 0.08821574]]
